# 📘 Ejemplo completo: Sensores + Spark + MariaDB + Jupyter

Este notebook demuestra cómo:
1. Conectarse desde Jupyter (PySpark) al clúster de Spark
2. Leer datos reales desde MariaDB (tabla `sensores`)
3. Visualizarlos en un gráfico con Matplotlib
4. Guardar la imagen en `/home/jovyan/work/shared/`

In [ ]:
from pyspark.sql import SparkSession

# Ruta del conector JDBC
jdbc_driver_path = "/opt/spark/jars/mariadb-java-client.jar"

# Crear sesión de Spark
spark = (
    SparkSession.builder
    .appName("LeerSensores")
    .master("spark://spark-master:7077")
    .config("spark.jars", jdbc_driver_path)
    .config("spark.driver.extraClassPath", jdbc_driver_path)
    .config("spark.executor.extraClassPath", jdbc_driver_path)
    .getOrCreate()
)

print("✅ SparkSession creada correctamente")
print("Versión:", spark.version)

In [ ]:
# --- Lectura de datos desde MariaDB ---
jdbc_url = "jdbc:mysql://mariadb:3306/bigdata_db?allowPublicKeyRetrieval=true&useSSL=false&permitMysqlScheme"

try:
    df = (
        spark.read.format("jdbc")
        .option("url", jdbc_url)
        .option("dbtable", "sensores")
        .option("user", "bigdata_user")
        .option("password", "bigdata_pass")
        .option("driver", "org.mariadb.jdbc.Driver")
        .load()
    )

    print("✅ Datos cargados correctamente desde MariaDB")
    df.printSchema()
    df.show(10)
except Exception as e:
    print("❌ Error al leer los datos:")
    print(e)

In [ ]:
# --- Gráfico de temperatura y humedad ---
import matplotlib.pyplot as plt
import pandas as pd

try:
    # Convertir a Pandas (solo las columnas necesarias)
    pdf = df.select("fecha", "temperatura", "humedad").toPandas()
    pdf = pdf.sort_values("fecha")

    plt.figure(figsize=(10, 5))
    plt.plot(pdf["fecha"], pdf["temperatura"], label="Temperatura (°C)", color="red")
    plt.plot(pdf["fecha"], pdf["humedad"], label="Humedad (%)", color="blue")
    plt.xlabel("Fecha")
    plt.ylabel("Valor")
    plt.title("Registro de Sensores")
    plt.legend()
    plt.grid(True)
    plt.show()
    print("✅ Gráfico generado correctamente")

except Exception as e:
    print("❌ Error al graficar:")
    print(e)

In [ ]:
# --- Guardar el gráfico en carpeta compartida ---
output_dir = "/home/jovyan/work/shared"
output_path = f"{output_dir}/grafico_sensores.png"

try:
    plt.figure(figsize=(10, 5))
    plt.plot(pdf["fecha"], pdf["temperatura"], label="Temperatura (°C)", color="red")
    plt.plot(pdf["fecha"], pdf["humedad"], label="Humedad (%)", color="blue")
    plt.xlabel("Fecha")
    plt.ylabel("Valor")
    plt.title("Registro de Sensores")
    plt.legend()
    plt.grid(True)
    plt.savefig(output_path, bbox_inches='tight')

    print(f"✅ Gráfico guardado en: {output_path}")

except Exception as e:
    print("❌ Error al guardar el gráfico:")
    print(e)

In [ ]:
spark.stop()